In [1]:
import torch
print(torch.cuda.is_available())  # 检查 CUDA 是否可用
print(torch.cuda.current_device())  # 查看当前选中的 GPU

True
0


/home/peterchen/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
from tqdm import tqdm
import json
import os
from datetime import datetime
import time
import logging
from utils import *
from config import parameters as conf
from torch import nn
import torch
import torch.optim as optim


from Model import Bert_model

stty: 'standard input': Inappropriate ioctl for device


In [4]:

from transformers import BertTokenizer
# 模型架构配置
from transformers import BertConfig
tokenizer = BertTokenizer.from_pretrained(conf.model_size)
model_config = BertConfig.from_pretrained(conf.model_size)

import json

model_dir_name = conf.model_save_name + "_" + \
    datetime.now().strftime("%Y%m%d%H%M%S")
model_dir = os.path.join(conf.output_path, model_dir_name)
results_path = os.path.join(model_dir, "results")
saved_model_path = os.path.join(model_dir, "saved_model")
os.makedirs(saved_model_path, exist_ok=False)
os.makedirs(results_path, exist_ok=False)
log_file = os.path.join(results_path, 'log.txt')

op_list = read_txt(conf.op_list_file, log_file)
op_list = [op + '(' for op in op_list]
op_list = ['EOF', 'UNK', 'GO', ')'] + op_list
const_list = read_txt(conf.const_list_file, log_file)
const_list = [const.lower().replace('.', '_') for const in const_list]
reserved_token_size = len(op_list) + len(const_list)

Reading: operation_list.txt
Reading: constant_list.txt


In [5]:
train_data, train_examples, op_list, const_list = \
read_partial_examples(input_path=conf.train_file, tokenizer=tokenizer,
        op_list=op_list, const_list=const_list, log_file=log_file,
        max_examples=100)

valid_data, valid_examples, op_list, const_list = \
read_partial_examples(input_path=conf.valid_file, tokenizer=tokenizer,
        op_list=op_list, const_list=const_list, log_file=log_file,
        max_examples=10)

test_data, test_examples, op_list, const_list = \
read_partial_examples(input_path=conf.test_file, tokenizer=tokenizer,
        op_list=op_list, const_list=const_list, log_file=log_file,
        max_examples=10)

kwargs = {"examples": train_examples,
"tokenizer": tokenizer,
"option": conf.option,
"is_training": True,
"max_seq_length": conf.max_seq_length,
}

Reading /home/peterchen/FinQA/dataset/train.json
Reading /home/peterchen/FinQA/dataset/dev.json
Reading /home/peterchen/FinQA/dataset/test.json


In [6]:
train_features = convert_examples_to_features(**kwargs)
kwargs["examples"] = valid_examples
kwargs["is_training"] = False
valid_features = convert_examples_to_features(**kwargs)
kwargs["examples"] = test_examples
test_features = convert_examples_to_features(**kwargs)

0it [00:00, ?it/s]

100it [00:08, 12.37it/s]
10it [00:00, 12.74it/s]
10it [00:00, 12.56it/s]


In [7]:
write_log(log_file, "####################INPUT PARAMETERS###################")
for attr in conf.__dict__:
    # .__dict__是一个特殊的属性，它包含了一个对象的所有属性和它们的值
    value = conf.__dict__[attr]
    write_log(log_file, attr + " = " + str(value))
write_log(log_file, "#######################################################")

model = Bert_model(hidden_size=model_config.hidden_size,
                       dropout_rate=conf.dropout_rate,)
# 多个GPU上并行
model = nn.DataParallel(model)
# 将模型发送到指定的设备
model.to(conf.device)
optimizer = optim.Adam(model.parameters(), conf.learning_rate)
# 使用交叉熵损失函数，忽略标签为-1的样本
criterion = nn.CrossEntropyLoss(reduction='none', ignore_index=-1)
model.train()

train_iterator = DataLoader(
        is_training=True, data=train_features, batch_size=conf.batch_size, shuffle=True)

####################INPUT PARAMETERS###################
__module__ = config
prog_name = retriever
root_path = /home/peterchen/FinQA/
output_path = /home/peterchen/FinQA/retriever_output/
cache_dir = /home/peterchen/FinQA/retriever_output/cache/
model_save_name = retriever-bert-base-test
train_file = /home/peterchen/FinQA/dataset/train.json
valid_file = /home/peterchen/FinQA/dataset/dev.json
test_file = /home/peterchen/FinQA/dataset/test.json
op_list_file = operation_list.txt
const_list_file = constant_list.txt
pretrained_model = bert
model_size = bert-base-uncased
device = cuda
mode = train
resume_model_path = 
saved_model_path = /home/peterchen/FinQA/retriever_output/bert-base-6k_20210427232814/saved_model/loads/3/model.pt
build_summary = False
option = rand
neg_rate = 3
topn = 5
sep_attention = True
layer_norm = True
num_decoder_layers = 1
max_seq_length = 512
max_program_length = 100
n_best_size = 20
dropout_rate = 0.1
batch_size = 16
batch_size_test = 16
epoch = 1
learning_rate = 2

In [8]:
# 初始化用于记录训练进度和性能的变量，如批次计数器、损失记录等
k = 0
record_k = 0
record_loss_k = 0
loss, start_time = 0.0, time.time()
record_loss = 0.0

print("train iter length", len(train_iterator))

train iter length 44


In [9]:
for first_batch in train_iterator:
    break

first_batch.keys()

dict_keys(['input_ids', 'input_mask', 'segment_ids', 'filename_id', 'label', 'ind'])

In [10]:
first_batch['ind']

['text_1',
 'table_2',
 'text_15',
 'table_1',
 'table_1',
 'table_8',
 'table_1',
 'table_6',
 'table_1',
 'table_2',
 'text_0',
 'text_1',
 'table_1',
 'table_15',
 'table_1',
 'text_7']

In [11]:
def evaluate(data_ori, data, model, ksave_dir, mode='valid'):
    # 评估模型在验证集（或测试集）上的性能
    # ksave_dir: 保存目录

    pred_list = []
    pred_unk = []

    ksave_dir_mode = os.path.join(ksave_dir, mode)
    os.makedirs(ksave_dir_mode, exist_ok=True)

    data_iterator = DataLoader(
        is_training=False, data=data, batch_size=conf.batch_size_test, shuffle=False)

    k = 0
    all_logits = []
    all_filename_id = []
    all_ind = []
    with torch.no_grad():
        # 关闭梯度计算
        for x in tqdm(data_iterator):

            input_ids = x['input_ids']
            input_mask = x['input_mask']
            segment_ids = x['segment_ids']
            label = x['label']
            filename_id = x["filename_id"]
            ind = x["ind"]

            ori_len = len(input_ids)
            for each_item in [input_ids, input_mask, segment_ids]:
                # 检查输入数据的长度，并在必要时对其进行填充，以确保每个批次的数据大小都与模型期望的一致
                if ori_len < conf.batch_size_test:
                    each_len = len(each_item[0])
                    pad_x = [0] * each_len
                    each_item += [pad_x] * (conf.batch_size_test - ori_len)

            input_ids = torch.tensor(input_ids).to(conf.device)
            input_mask = torch.tensor(input_mask).to(conf.device)
            segment_ids = torch.tensor(segment_ids).to(conf.device)

            # 使用模型进行预测，得到logits（模型的原始输出，通常在应用softmax之前的值）
            logits = model(True, input_ids, input_mask,
                           segment_ids, device=conf.device)

            # 将所有预测结果、文件名标识符、索引存储起来
            all_logits.extend(logits.tolist())
            all_filename_id.extend(filename_id)
            all_ind.extend(ind)

    output_prediction_file = os.path.join(ksave_dir_mode,
                                          "predictions.json")
    
    # 预测结果被写入到predictions.json文件中
    if mode == "valid":
        print_res = retrieve_evaluate(
            all_logits, all_filename_id, all_ind, output_prediction_file, conf.valid_file, topn=conf.topn)
    else:
        print_res = retrieve_evaluate(
            all_logits, all_filename_id, all_ind, output_prediction_file, conf.test_file, topn=conf.topn)

    write_log(log_file, print_res)
    print(print_res)
    return

In [ ]:
for x in train_iterator:
            
    # print("current batch: ", k)
    # start_time = time.time()

    input_ids = torch.tensor(x['input_ids']).to(conf.device)
    input_mask = torch.tensor(x['input_mask']).to(conf.device)
    segment_ids = torch.tensor(x['segment_ids']).to(conf.device)
    label = torch.tensor(x['label']).to(conf.device)

    # 清除之前的梯度
    model.zero_grad()
    optimizer.zero_grad()

    this_logits = model(True, input_ids, input_mask,
                        segment_ids, device=conf.device)

    # print("complete inference: ", time.time() - start_time)
    # start_time = time.time()        

    # 计算损失
    this_loss = criterion(
        this_logits.view(-1, this_logits.shape[-1]), label.view(-1))

    this_loss = this_loss.sum()
    record_loss += this_loss.item() * 100
    record_k += 1
    k += 1

    # 执行反向传播，更新模型参数
    this_loss.backward()
    optimizer.step()

    # print("complete backward: ", time.time() - start_time)

    if k > 1 and k % conf.report_loss == 0:
        # 每处理一定数量的批次后，记录当前的平均损失
        write_log(log_file, "%d : loss = %.3f" %
                    (k, record_loss / record_k))
        record_loss = 0.0
        record_k = 0

    if k > 1 and k % conf.report == 0:
        # 每隔一定数量的批次，评估模型在验证集上的性能，并保存当前的模型
        print("Round: ", k / conf.report)
        model.eval()
        cost_time = time.time() - start_time
        write_log(log_file, "%d : time = %.3f " %
                    (k // conf.report, cost_time))
        start_time = time.time()
        if k // conf.report >= 1:
            print("Val test")
            # save model
            saved_model_path_cnt = os.path.join(
                saved_model_path, 'loads', str(k // conf.report))
            os.makedirs(saved_model_path_cnt, exist_ok=True)
            torch.save(model.state_dict(),
                        saved_model_path_cnt + "/model.pt")
            # .state_dict() 是一个包含整个模型的参数和持久化缓冲区（比如批量归一化的运行平均值）的Python字典
            # .pt 或 .pth 文件是PyTorch的标准文件扩展名，用于保存模型的参数

            results_path_cnt = os.path.join(
                results_path, 'loads', str(k // conf.report))
            os.makedirs(results_path_cnt, exist_ok=True)
            validation_result = evaluate(
                valid_examples, valid_features, model, results_path_cnt, 'valid')
            write_log(log_file, validation_result)

        model.train()

KeyboardInterrupt: 

In [2]:
import json
p = '/home/peterchen/FinQA/dataset/dev copy.json'
json.loads(p)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)